# Database Validation - Comprehensive Health Check

## Purpose
Run comprehensive validation checks on the FAQ tracking database to ensure:
- Schema integrity (tables, views, indexes, triggers)
- Data integrity (foreign keys, checksums, timestamps)
- Business logic compliance
- Performance optimization

## When to Run
- After setting up database with `1_setup_tables_STREAMLINED.ipynb`
- After major code or schema changes
- Before deploying to production
- When debugging data issues
- Periodically during development

## What It Checks
Executes comprehensive validation queries from `sql/validation.sql`:
1. **Schema Validation**: Tables, views, indexes, triggers
2. **Timestamp Format**: ISO-8601 UTC compliance
3. **Checksum Validation**: SHA-256 format compliance
4. **Foreign Key Integrity**: Referential integrity checks
5. **Trigger Functionality**: Diff enforcement logic
6. **View Accessibility**: All views queryable
7. **Data Integrity**: Business logic compliance
8. **Performance**: Index usage and optimization

## Output
- Color-coded results (PASS/WARN/FAIL)
- Summary dashboard by category
- Detailed validation report (exported to text file)

In [ ]:
# Setup and Imports
import sys
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime
from IPython.display import HTML, display

# Paths
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
DATABASE_DIR = PROJECT_ROOT / 'databases'
DATABASE_NAME = 'faq_update.db'
DB_PATH = DATABASE_DIR / DATABASE_NAME
VALIDATION_SQL_PATH = PROJECT_ROOT / 'sql' / 'validation.sql'

# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

print(f"📁 Database: {DB_PATH}")
print(f"📄 Validation SQL: {VALIDATION_SQL_PATH}")
print(f"🕐 Validation Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verify files exist
if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found: {DB_PATH}\n" + 
                           f"Run 1_setup_tables_STREAMLINED.ipynb first")
if not VALIDATION_SQL_PATH.exists():
    raise FileNotFoundError(f"Validation SQL not found: {VALIDATION_SQL_PATH}")

print("\n✅ Setup complete")

In [ ]:
# Load and Execute Validation SQL
print("📊 Loading validation SQL...")

with open(VALIDATION_SQL_PATH, 'r', encoding='utf-8') as f:
    validation_sql = f.read()

# Split SQL into individual queries (separated by semicolons)
# Note: This is a simple split - assumes no semicolons in strings
queries = [q.strip() for q in validation_sql.split(';') if q.strip() and not q.strip().startswith('--')]

print(f"Found {len(queries)} validation queries")

# Execute all validation queries
results = []
with sqlite3.connect(DB_PATH) as conn:
    for idx, query in enumerate(queries):
        # Skip PRAGMA statements and comments
        if query.upper().startswith('PRAGMA') or query.startswith('--'):
            continue
        
        try:
            df = pd.read_sql(query, conn)
            if not df.empty:
                results.append(df)
        except Exception as e:
            print(f"⚠️  Query {idx+1} failed: {str(e)[:100]}")

print(f"\n✅ Executed validation queries, got {len(results)} result sets")

In [ ]:
# Parse and Organize Results by Category
print("🔍 Parsing validation results...\n")

# Combine all results into single DataFrame
if results:
    all_results = pd.concat(results, ignore_index=True)
    
    # Standardize column names (handle different query formats)
    # Most queries return: check_category, check_name, status, ...
    if 'check_category' in all_results.columns:
        validation_df = all_results
    else:
        # Handle queries that don't follow standard format
        validation_df = all_results
        if 'status' not in validation_df.columns:
            validation_df['status'] = 'UNKNOWN'
        if 'check_category' not in validation_df.columns:
            validation_df['check_category'] = 'General'
        if 'check_name' not in validation_df.columns:
            validation_df['check_name'] = 'Unnamed Check'
    
    # Group by category
    categories = validation_df['check_category'].unique()
    print(f"Found {len(categories)} validation categories:")
    for cat in categories:
        count = len(validation_df[validation_df['check_category'] == cat])
        print(f"  - {cat}: {count} checks")
    
    print(f"\n✅ Parsed {len(validation_df)} validation checks")
else:
    print("⚠️  No validation results found")
    validation_df = pd.DataFrame()

In [ ]:
# Display Results with Color-Coding
def color_status(val):
    """Color-code status values: PASS=green, WARN=yellow, FAIL=red"""
    if pd.isna(val):
        return ''
    val_upper = str(val).upper()
    if 'PASS' in val_upper:
        return 'background-color: #d4edda; color: #155724'  # Green
    elif 'WARN' in val_upper:
        return 'background-color: #fff3cd; color: #856404'  # Yellow
    elif 'FAIL' in val_upper:
        return 'background-color: #f8d7da; color: #721c24'  # Red
    else:
        return ''

if not validation_df.empty:
    print("="*100)
    print("VALIDATION RESULTS")
    print("="*100)
    
    # Display results by category
    for category in sorted(validation_df['check_category'].unique()):
        print(f"\n{'='*100}")
        print(f"CATEGORY: {category}")
        print(f"{'='*100}")
        
        cat_df = validation_df[validation_df['check_category'] == category]
        
        # Display with color coding
        styled_df = cat_df.style.applymap(color_status, subset=['status'])
        display(styled_df)
        
        # Show summary for this category
        if 'status' in cat_df.columns:
            status_counts = cat_df['status'].value_counts()
            print(f"\nCategory Summary:")
            for status, count in status_counts.items():
                print(f"  {status}: {count}")
else:
    print("⚠️  No validation results to display")

In [ ]:
# Generate Summary Dashboard
if not validation_df.empty and 'status' in validation_df.columns:
    print("\n" + "="*100)
    print("VALIDATION SUMMARY DASHBOARD")
    print("="*100)
    
    # Overall summary
    total_checks = len(validation_df)
    pass_count = len(validation_df[validation_df['status'].str.upper().str.contains('PASS', na=False)])
    warn_count = len(validation_df[validation_df['status'].str.upper().str.contains('WARN', na=False)])
    fail_count = len(validation_df[validation_df['status'].str.upper().str.contains('FAIL', na=False)])
    
    print(f"\n📊 Overall Results:")
    print(f"  Total Checks:    {total_checks}")
    print(f"  ✅ PASS:         {pass_count} ({pass_count/total_checks*100:.1f}%)")
    print(f"  ⚠️  WARN:         {warn_count} ({warn_count/total_checks*100:.1f}%)")
    print(f"  ❌ FAIL:         {fail_count} ({fail_count/total_checks*100:.1f}%)")
    
    # Summary by category
    print(f"\n📋 Results by Category:")
    summary_by_cat = validation_df.groupby(['check_category', 'status']).size().unstack(fill_value=0)
    print(summary_by_cat)
    
    # Health score
    health_score = (pass_count / total_checks * 100) if total_checks > 0 else 0
    print(f"\n🏥 Database Health Score: {health_score:.1f}%")
    
    if health_score == 100:
        print("   Status: 🟢 EXCELLENT - All checks passed!")
    elif health_score >= 90:
        print("   Status: 🟡 GOOD - Minor issues detected")
    elif health_score >= 70:
        print("   Status: 🟠 WARNING - Multiple issues require attention")
    else:
        print("   Status: 🔴 CRITICAL - Significant issues found!")
    
    # Failed/Warning checks details
    if fail_count > 0:
        print(f"\n❌ FAILED CHECKS ({fail_count}):")
        failed = validation_df[validation_df['status'].str.upper().str.contains('FAIL', na=False)]
        for idx, row in failed.iterrows():
            print(f"  - [{row['check_category']}] {row['check_name']}")
    
    if warn_count > 0:
        print(f"\n⚠️  WARNING CHECKS ({warn_count}):")
        warnings = validation_df[validation_df['status'].str.upper().str.contains('WARN', na=False)]
        for idx, row in warnings.iterrows():
            print(f"  - [{row['check_category']}] {row['check_name']}")
    
    print("\n" + "="*100)
else:
    print("⚠️  Cannot generate summary - no validation data")

In [ ]:
# Export Validation Report to Text File
report_filename = f"validation_report_{datetime.now().strftime('%Y-%m-%d_%H%M%S')}.txt"
report_path = DATABASE_DIR / report_filename

print(f"📝 Exporting validation report to: {report_path}")

with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*100 + "\n")
    f.write("FAQ UPDATE DATABASE - VALIDATION REPORT\n")
    f.write("="*100 + "\n")
    f.write(f"Database: {DB_PATH}\n")
    f.write(f"Validation Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*100 + "\n\n")
    
    if not validation_df.empty:
        # Overall summary
        total_checks = len(validation_df)
        pass_count = len(validation_df[validation_df['status'].str.upper().str.contains('PASS', na=False)])
        warn_count = len(validation_df[validation_df['status'].str.upper().str.contains('WARN', na=False)])
        fail_count = len(validation_df[validation_df['status'].str.upper().str.contains('FAIL', na=False)])
        health_score = (pass_count / total_checks * 100) if total_checks > 0 else 0
        
        f.write("SUMMARY\n")
        f.write("-" * 100 + "\n")
        f.write(f"Total Checks: {total_checks}\n")
        f.write(f"PASS: {pass_count} ({pass_count/total_checks*100:.1f}%)\n")
        f.write(f"WARN: {warn_count} ({warn_count/total_checks*100:.1f}%)\n")
        f.write(f"FAIL: {fail_count} ({fail_count/total_checks*100:.1f}%)\n")
        f.write(f"Health Score: {health_score:.1f}%\n")
        f.write("\n")
        
        # Detailed results by category
        f.write("DETAILED RESULTS\n")
        f.write("=" * 100 + "\n\n")
        
        for category in sorted(validation_df['check_category'].unique()):
            f.write(f"\nCATEGORY: {category}\n")
            f.write("-" * 100 + "\n")
            cat_df = validation_df[validation_df['check_category'] == category]
            f.write(cat_df.to_string(index=False))
            f.write("\n\n")
        
        # Failed/Warning details
        if fail_count > 0:
            f.write("\nFAILED CHECKS\n")
            f.write("=" * 100 + "\n")
            failed = validation_df[validation_df['status'].str.upper().str.contains('FAIL', na=False)]
            f.write(failed.to_string(index=False))
            f.write("\n\n")
        
        if warn_count > 0:
            f.write("\nWARNING CHECKS\n")
            f.write("=" * 100 + "\n")
            warnings = validation_df[validation_df['status'].str.upper().str.contains('WARN', na=False)]
            f.write(warnings.to_string(index=False))
            f.write("\n\n")
    else:
        f.write("No validation results found.\n")
    
    f.write("\n" + "="*100 + "\n")
    f.write("END OF REPORT\n")
    f.write("="*100 + "\n")

print(f"\n✅ Validation report exported successfully")
print(f"📄 Report location: {report_path}")
print(f"\nTo view report:\n  cat {report_path}")
print(f"\nTo find issues:\n  grep -E '(FAIL|WARN)' {report_path}")